# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import datasets
from sklearn import linear_model
import os

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
%matplotlib inline


# iowa_data = pd.read_csv('/Users/HudsonCavanagh/Documents/Iowa_Liquor_Sales.csv') -- this is where need to update with full dataset (eventually)
iowa_data = pd.read_csv('/Users/HudsonCavanagh/Documents/Iowa_Liquor_sales_sample_10pct.csv')
ia = pd.DataFrame(iowa_data) 


iowa_zip_data = pd.read_csv('/Users/HudsonCavanagh/dsi-projects/projects/3rd - IA Liquor/iowa_zip_pop.csv')
ia_zip_pop = pd.DataFrame(iowa_zip_data) 



ia['Date'] = pd.to_datetime(ia['Date'], infer_datetime_format=True)

ia_zip_pop.head()

,url,Reportdata number 1,Reportdata link 1,Reportdata link 1_link,Reportdata number 2,Reportdata link 2,Reportdata link 2_link,Reportdata number 3,Reportdata number 4,Reportdata number 5,Unnamed: 10
0,http://zipatlas.com/us/ia/zip-code-comparison/...,101,50009,http://zipatlas.com/us/ia/altoona/zip-50009.htm,"41.640625, -93.458835",Altoona,http://zipatlas.com/us/ia/altoona.htm,"12,008",406.63,"#8,763",407
1,http://zipatlas.com/us/ia/zip-code-comparison/...,102,51451,http://zipatlas.com/us/ia/lanesboro/zip-51451.htm,"42.184080, -94.695158",Lanesboro,http://zipatlas.com/us/ia/lanesboro.htm,148,401.19,"#8,801",401
2,http://zipatlas.com/us/ia/zip-code-comparison/...,103,52002,http://zipatlas.com/us/ia/dubuque/zip-52002.htm,"42.521344, -90.776310",Dubuque,http://zipatlas.com/us/ia/dubuque.htm,"11,539",400.73,"#8,806",401
3,http://zipatlas.com/us/ia/zip-code-comparison/...,104,50706,http://zipatlas.com/us/ia/waterloo/zip-50706.htm,"42.408728, -92.258971",Waterloo,http://zipatlas.com/us/ia/waterloo.htm,"1,035",395.19,"#8,843",395
4,http://zipatlas.com/us/ia/zip-code-comparison/...,105,51355,http://zipatlas.com/us/ia/okoboji/zip-51355.htm,"43.388189, -95.136678",Okoboji,http://zipatlas.com/us/ia/okoboji.htm,825,388.07,"#8,902",388


In [2]:
#clean IA ZIP POP HERE

# ia_zip_pop = ia_zip_pop.loc['Reportdata link 1', 'Reportdata number 2', 'Reportdata link 2', 'Reportdata number 4']
##ORIGINAL VERSION:
# ia_zip_pop = ia_zip_pop.iloc[:,(2,4,5,8)]
# column_iazip = ['zip', 'lat_long', 'city', 'pop_dense_heads_sqm']

#REVISED FOR MERGING

ia_zip_pop = ia_zip_pop.iloc[:,(2,10)]
column_iazip = ['zip', 'pop_dense_heads_sqm']
ia_zip_pop.columns = column_iazip
# ia_zip_pop.dropna(inplace=True)
ia_zip_pop['zip'] = ia_zip_pop['zip'].apply(lambda x: str(x))
ia_zip_pop['pop_dense_heads_sqm'] = ia_zip_pop['pop_dense_heads_sqm'].apply(lambda x: float(x)) 
#could not get the above to read commas in numbers over 1k with float or int, did conversion in google sheets

In [3]:
ia_zip_pop.head()



,zip,pop_dense_heads_sqm
0,50009,407
1,51451,401
2,52002,401
3,50706,395
4,51355,388


In [4]:
#Need to look into regularization, standardization and min-max adjustment for each columns
##PLAN FOR EACH COLUMN

# Notes on original data presentation

#   Invoice/Item Number  --- unique ID, leave, should be string
#   Date  --- need to convert pd.DateTime()
#   Store Number --- use GROUPBY() on this
#   Store Name -- see above, string
#   Address -- see above, string
#   City -- should be used in pivot table indexing
#   Zip Code -- GROUPBY candidate & should be used in pivottable indexing
#   Store Location - can strip long & lat out of this
#   County Number - potential groupby?
#   County - see above 
#   Category -- see below (can be used for groupby)
#   Category Name -- type of liquor sold
#   Vendor Number - seller ID
#   Vendor Name - seller (see above)
#   Item Number - specific product ID (groupby)
#   Item Description - product ID descrip
#   Pack - # bottles sold per sale
#   Bottle Volume (ml)
#   State Bottle Cost
#   State Bottle Retail
#   Bottles Sold - number of units sold
#   Sale (Dollars)
#   Volume Sold (Liters)
#   Volume Sold (Gallons)

In [5]:


ia.rename(columns={'Date': 'date', 'Store Number': 'store_num', 'City': 'city', 'Zip Code': 'zip', 'County Number': 'county_num', 'County': 'county_name', 'Category': 'cat', 'Category Name': 'cat_name'}, inplace=True)
ia.rename(columns={'Vendor Number': 'vend_id', 'Item Number': 'item_id', 'Item Description': 'item', 'Bottle Volume (ml)': 'vol_per_bottle_ml', 'State Bottle Cost': 'bottle_cost', 'State Bottle Retail': 'retail_unit_rev', 'Bottles Sold': 'bottles_sold'}, inplace=True)
ia.rename(columns={'Sale (Dollars)': 'trans_revenue', 'Volume Sold (Liters)': 'vol_sold_liters'}, inplace=True)
ia = ia.iloc[:,0:17]

ia.head()

,date,store_num,city,zip,county_num,county_name,cat,cat_name,vend_id,item_id,item,vol_per_bottle_ml,bottle_cost,retail_unit_rev,bottles_sold,trans_revenue,vol_sold_liters
0,2015-11-04,3717,SUMNER,50674,9,Bremer,1051100,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0
1,2016-03-02,2614,DAVENPORT,52807,82,Scott,1011100,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5
2,2016-02-11,2106,CEDAR FALLS,50613,7,Black Hawk,1011200,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0
3,2016-02-03,2501,AMES,50010,85,Story,1071100,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5
4,2015-08-18,3654,BELMOND,50421,99,Wright,1031080,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0


In [6]:
#CONVERT DATATYPES

# ia['date'].value_counts  #dtype time 64
# ia['store_num'].value_counts  #int, should be string
ia['store_num'] = ia['store_num'].apply(lambda x: str(x))
# ia['city'].value_counts #object, ok
# ia['zip'].value_counts #object, ok
# ia['county_num'].value_counts #float, should be object
ia['county_num'] = ia['county_num'].apply(lambda x: str(x))
# ia['county_name'].value_counts #object, ok
# ia['cat'].value_counts #int, should be obj
                                          
ia['cat'] = ia['cat'].apply(lambda x: str(x))
                                          
                                 
# ia['cat_name'].value_counts #str
ia['vend_id'] = ia['vend_id'].apply(lambda x: str(x))
ia['item_id'] = ia['item_id'].apply(lambda x: str(x))
# ia['item'].value_counts #str
ia['liters_per_bottle'] = (ia['vol_per_bottle_ml'].apply(lambda x: float(x)/1000))
ia['bottles_sold'] = (ia['bottles_sold'].apply(lambda x: float(x)))

##THIS DATA CLEAN, JUST NEED TO CONVERT DOLLAR COLUMNS

ia['bottle_cost'] = ia['bottle_cost'].apply(lambda x: x.strip('$'))
ia['bottle_cost'] = ia['bottle_cost'].apply(lambda x: float(x))

ia['retail_unit_rev'] = ia['retail_unit_rev'].apply(lambda x: x.strip('$'))
ia['retail_unit_rev'] = ia['retail_unit_rev'].apply(lambda x: float(x))

ia['trans_revenue'] = ia['trans_revenue'].apply(lambda x: x.strip('$'))
ia['trans_revenue'] = ia['trans_revenue'].apply(lambda x: float(x))


ia.head()



,date,store_num,city,zip,county_num,county_name,cat,cat_name,vend_id,item_id,item,vol_per_bottle_ml,bottle_cost,retail_unit_rev,bottles_sold,trans_revenue,vol_sold_liters,liters_per_bottle
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0,0.75
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.75
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,1.00
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.5,1.75
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.0,1.75


In [7]:
#add calculated features

ia['profit'] = (ia['retail_unit_rev']-ia['bottle_cost'])*ia['bottles_sold']
ia['profit_per_L'] = ia['profit']/ia['liters_per_bottle']
ia['profit_margin'] = ia['profit']/ia['trans_revenue']



ia.head()


,date,store_num,city,zip,county_num,county_name,cat,cat_name,vend_id,item_id,...,vol_per_bottle_ml,bottle_cost,retail_unit_rev,bottles_sold,trans_revenue,vol_sold_liters,liters_per_bottle,profit,profit_per_L,profit_margin
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,...,750,4.50,6.75,12,81.00,9.0,0.75,27.00,36.000000,0.333333
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,...,750,13.75,20.63,2,41.26,1.5,0.75,13.76,18.346667,0.333495
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,...,1000,12.59,18.89,24,453.36,24.0,1.00,151.20,151.200000,0.333510
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,...,1750,9.50,14.25,6,85.50,10.5,1.75,28.50,16.285714,0.333333
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,...,1750,7.20,10.80,12,129.60,21.0,1.75,43.20,24.685714,0.333333


In [8]:

ia_pop = pd.merge(ia, ia_zip_pop, how='left', on='zip', sort=False).head(50)

        
ia_pop.head()



,date,store_num,city,zip,county_num,county_name,cat,cat_name,vend_id,item_id,...,bottle_cost,retail_unit_rev,bottles_sold,trans_revenue,vol_sold_liters,liters_per_bottle,profit,profit_per_L,profit_margin,pop_dense_heads_sqm
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,...,4.50,6.75,12,81.00,9.0,0.75,27.00,36.000000,0.333333,24
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,...,13.75,20.63,2,41.26,1.5,0.75,13.76,18.346667,0.333495,1252
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,...,12.59,18.89,24,453.36,24.0,1.00,151.20,151.200000,0.333510,302
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,...,9.50,14.25,6,85.50,10.5,1.75,28.50,16.285714,0.333333,419
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,...,7.20,10.80,12,129.60,21.0,1.75,43.20,24.685714,0.333333,35


In [20]:

# g = ia_pop.groupby(pd.TimeGrouper('M', closed = 'left')).aggregate(numpy.sum)

# store_ia = g.groupby(['store_num']) #should already be grouped by month at this point
# store_ia.head()

g_date_storeQ = ia_pop.set_index('date').groupby(pd.TimeGrouper('Q')).apply(lambda x: x.groupby('store_num').sum())
g_date_storeM = ia_pop.set_index('date').groupby(pd.TimeGrouper('M')).apply(lambda x: x.groupby('store_num').sum())
#train-test-split this 

g_store = ia_pop.set_index('store_num')
# grouped_store = g_store.groupby(lambda x: x.month)
# store_sales = grouped_store.aggregate(np.sum)


# g_store.head()

g_date_storeQ_X = g_date_storeQ.drop('trans_revenue', 1)
g_date_storeQ_y = g_date_storeQ['trans_revenue']


Q1_2015_y = g_date_storeQ_y.loc['2015-03-31']

Q2_2015_y = g_date_storeQ_y.loc['2015-06-30']
Q3_2015_y = g_date_storeQ_y.loc['2015-09-30']
Q4_2015_y = g_date_storeQ_y.loc['2015-12-31']
Q1_2016_y = g_date_storeQ_y.loc['2016-03-31']

Q1_2015_X = g_date_storeQ_X.loc['2015-03-31']
Q2_2015_X = g_date_storeQ_X.loc['2015-06-30']
Q3_2015_X = g_date_storeQ_X.loc['2015-09-30']
Q4_2015_X = g_date_storeQ_X.loc['2015-12-31']
Q1_2016_X = g_date_storeQ_X.loc['2016-03-31']

# g_date_storeQ_X.head()

frames = [Q2_2015_y, Q3_2015_y, Q4_2015_y]
Q23_2015_Y = pd.merge(Q2_2015_y, Q3_2015_y, how='inner', left_index=True, right_index=True)







print(Q23_2015_Y)



#SAM: train on Q1 data in order to evaluate Q2,3,4 in 2015, 
#then use the model used to predict Q2,3,4 in 2015 on the same in 2016

IndexError: list index out of range

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [21]:

Q3_2015_X

,vol_per_bottle_ml,bottle_cost,retail_unit_rev,bottles_sold,vol_sold_liters,liters_per_bottle,profit,profit_per_L,profit_margin,pop_dense_heads_sqm
store_num,,,,,,,,,,
2596,750,6.66,9.99,3,2.25,0.75,9.99,13.320000,0.333333,132
2648,750,26.24,39.36,6,4.50,0.75,78.72,104.960000,0.333333,1673
3654,1750,7.20,10.80,12,21.00,1.75,43.20,24.685714,0.333333,35
3713,1750,7.11,10.67,6,10.50,1.75,21.36,12.205714,0.333646,202


In [23]:
g_date_storeQ

vol_per_bottle_ml  bottle_cost  retail_unit_rev  \
date       store_num                                                    
2015-03-31 2514                     750         5.48             8.22   
           2545                     500         4.90             7.35   
           2549                    1750         7.20            10.80   
           2555                     750        12.07            18.11   
           2590                     750        14.25            21.38   
           2614                    1750         9.97            14.96   
           2644                    1000         7.62            11.43   
           3390                    1000        17.90            26.85   
           3612                    1000         5.28             7.92   
           3808                    1000         7.62            11.43   
           3858                    1750         8.92            13.38   
           3942                     750         5.23             7.85   
           3981                     750         7.00            10.50   
2015-06-30 2532                    1000         5.28             7.92   
           2569                    1750        13.32            19.98   
           2626                     375         6.52             9.78   
           3456                    1750        15.75            23.63   
           3717                     750         4.50             6.75   
           3963                     750         3.50             5.25   
           4195                     750         5.38             8.07   
           4236                    1000        11.75            17.63   
           4298                    1750         7.02            10.53   
           4438                     375         1.99             2.99   
           4520                     750        12.25            18.38   
2015-09-30 2596                     750         6.66             9.99   
           2648                     750        26.24            39.36   
           3654                    1750         7.20            10.80   
           3713                    1750         7.11            10.67   
2015-12-31 2502                     750        39.50            59.25   
           2600                    2500        12.50            19.00   
           3565                     300         7.35            11.03   
           3717                     750         4.50             6.75   
           4346                    1000         7.62            11.43   
           4375                    1750        10.99            16.49   
           4376                     750        10.49            15.74   
           4604                    1000         4.92             7.38   
           4674                     750         6.90            10.35   
2016-03-31 2106                    1000        12.59            18.89   
           2130                     200         1.13             1.70   
           2501                    1750         9.50            14.25   
           2614                     750        13.75            20.63   
           2618                     750         3.37             5.06   
           2633                    1000        16.16            24.24   
           2637                     375         6.50             9.75   
           2670                    1000         7.50            11.25   
           3635                    1000        18.75            28.13   
           3678                     750         8.26            12.39   
           4673                     750        10.00            15.00   
           4757                     750        11.49            17.24   

                      bottles_sold  trans_revenue  vol_sold_liters  \
date       store_num                                                 
2015-03-31 2514                  4          32.88             3.00   
           2545                  1           7.35             0.50   
           2549   

## Record your findings

Be sure to write out anything observations from your exploratory analysis.

# Mine the data
Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to
compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html) for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.

In [10]:
Q1_2015_X

,vol_per_bottle_ml,bottle_cost,retail_unit_rev,bottles_sold,vol_sold_liters,liters_per_bottle,profit,profit_per_L,profit_margin,pop_dense_heads_sqm
store_num,,,,,,,,,,
2514,750,5.48,8.22,4,3.00,0.75,10.96,14.613333,0.333333,34
2545,500,4.90,7.35,1,0.50,0.50,2.45,4.900000,0.333333,2290
2549,1750,7.20,10.80,12,21.00,1.75,43.20,24.685714,0.333333,101
2555,750,12.07,18.11,2,1.50,0.75,12.08,16.106667,0.333517,246
2590,750,14.25,21.38,2,1.50,0.75,14.26,19.013333,0.333489,2743
2614,1750,9.97,14.96,6,10.50,1.75,29.94,17.108571,0.333556,1252
2644,1000,7.62,11.43,12,12.00,1.00,45.72,45.720000,0.333333,194
3390,1000,17.90,26.85,24,24.00,1.00,214.80,214.800000,0.333333,43
3612,1000,5.28,7.92,3,3.00,1.00,7.92,7.920000,0.333333,55


In [11]:
Q234_2015_Y

,trans_revenue,trans_revenue,trans_revenue
2502,NaN,NaN,118.50
2532,95.04,NaN,NaN
2569,119.88,NaN,NaN
2596,NaN,29.97,NaN
2600,NaN,NaN,172.50
2626,39.12,NaN,NaN
2648,NaN,236.16,NaN
3456,47.26,NaN,NaN
3565,NaN,NaN,11.03
3654,NaN,129.60,NaN


# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

In [12]:
# MAKE TONS OF PLOTS, seaborn

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(Q1_2015_X, Q234_2015_Y, test_size=.33)


print "       X Shape  Y Shape"
print "Train", X_train.shape, y_train.shape
print "Test ", X_test.shape, y_test.shape


ValueError: Found arrays with inconsistent numbers of samples: [13 23]

In [ ]:
from sklearn import linear_model


lm = linear_model.LinearRegression()

X = ia[["RM, other col, other col, other col"]]
y = targets["MEDV"]

model = lm.fit(X, y)
predictions = lm.predict(X)


In [ ]:
# How is it performing? Plot the model's predictions against actual values
# s = s: size in points, c = color, zorder = layer order

plt.figure(figsize=(16,8))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from RM")
plt.ylabel("Actual Values MEDV")
plt.show()
print "MSE:", mean_squared_error(y, predictions)

cross_val_score(lr, X, y, n_jobs=1, cv=5).mean() #multiple jobs

In [ ]:
#plot the residuals

plt.figure(figsize=(16,8))
plt.scatter(y, y - predictions, c = 'b', marker = '+') # Look directly at the residuals
plt.axhline(0, color='r') #from http://localhost:8888/notebooks/week03/W3%20L1.3_scikit-modeling.ipynb

In [ ]:
#OLS

# Note the difference in argument order
OLSmodel = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Plot the model
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from RM")
plt.ylabel("Actual Values MEDV")
plt.show()
print "MSE:", mean_squared_error(y, predictions)

cross_val_score(lr, X, y, n_jobs=1, cv=5).mean() #multiple jobs

In [ ]:
print(OLSmodel.summary())

In [ ]:
#standardized Ridge Model

rcv = linear_model.RidgeCV(alphas=
                           (.001, .001, .01, .1, .5, 1, 5, 10),
                           store_cv_values=True)
rcv_model = rcv.fit(X_train, y_train)
y_predicted = rcv_model.predict(X_test)
rcv_r2 =  r2_score(y_true=y_test, y_pred=y_predicted)
rcv_r2

plt.scatter(y_test, y_predicted) #compare the hold-out observed y values to the predictions (made with model from test data)
plt.xlabel("True Values")
plt.ylabel("Predictions")

print "Score:", rcv_model.score(X_test, y_test)
rcv_model.cv_values_.mean()

rcv_model.alpha_ #this returns optimal alpha for dataset

cross_val_score(lr, X, y, n_jobs=1, cv=5).mean() #multiple jobs

In [ ]:
#standardized lasso model

lcv = linear_model.LassoCV()
lcv_model = lcv.fit(X_train, y_train)
y_lasso_predicted = lcv_model.predict(X_test)
lcv_r2 =  r2_score(y_true=y_test, y_pred=y_predicted)
lcv_r2

plt.scatter(y_test, y_lasso_predicted) #compare the hold-out observed y values to the predictions (made with model from test data)
plt.xlabel("True Values")
plt.ylabel("Lasso Predictions")

print "Score:", lcv_model.score(X_test, y_test)

print(lcv_model.alpha_, #this returns optimal alpha for dataset
abs(lcv_model.coef_))

final_predicts = ia.predict(modelname, X)

In [ ]:
#iterate through r2 or mse a bunch

for i in range(1000):
    cross_val_list = []
    n = cross_val_score(lasso, X, y, n_jobs=1, cv=5, scoring='r2').mean()
    cross_val_list.append(n)
print(np.mean(cross_val_list))


# cross_val_score(lr, X, y, n_jobs=1, cv=5,
#                 scoring='mean_squared_error').mean()

In [ ]:
# sample code for cross_validated output


X_train, X_test, y_train, y_test = train_test_split(dd, dy, test_size=.4)

lcv_model = lcv.fit(X_train, y_train)
lcv_pred = lcv.predict(X_test)
lasso_r2 =  r2_score(y_true=y_test, y_pred=lcv_pred)
print("R sq for Lasso Reg is:", lasso_r2)

print("cross validated r^2:", np.mean(cross_val_score(lcv_model, X_test, y_test, scoring='r2', cv=5)))
print("r^2 w/o cross-validation", (lcv_model.score(X_test,y_test)))
print("cross validated MSE (sign flipped):", -np.mean(cross_val_score(lcv_model, X_test, y_test, scoring='mean_squared_error', cv=5)))
print("MSE w/o cross-validation", (lcv_model.score(X_test,y_test)))

# mse
# bar
# bias

In [ ]:
###LOSS FUNCTIONS

from sklearn.metrics import mean_squared_error, mean_absolute_error
print "RMSE:", mean_squared_error(ys, predictions)
print "MAE:", mean_absolute_error(ys, predictions)

In [ ]:
#QUANT REG

df = pd.DataFrame(np.array([xs, ys]).transpose(), columns=["x", "y"])
df.columns = ["x", "y"]
mod = smf.quantreg('y ~ x', df)
res = mod.fit(q=.5)
print(res.summary())




In [ ]:
#HELPFUL FUNCTIONS

# Gradient Descent/ Optimizing Functions:
    
    
def mean_squared_error(y_true, x, beta0, beta1):
    y_pred = beta0 + x * beta1
    mean_sq_err = np.mean((y_true - y_pred)**2)
    return mean_sq_err

def gradient_update(y, x, beta0, beta1, step_size):
    
    beta0_gradient = 0
    beta1_gradient = 0
    
    N = float(len(y))
    
    for i in range(len(y)):
        
        # add to the beta0 gradient for each x,y using the partial derivative with respect to beta0
        beta0_gradient += (2./N * -1 * (y[i] - (beta0 + beta1*x[i])))
        
        # add to the beta1 gradient for each x,y using the partial derivative with respect to beta1
        beta1_gradient += (2./N * -1 * x[i] * (y[i] - (beta0 + beta1*x[i])))
        
    # update beta0 and beta1:
    beta0 = beta0 - (step_size * beta0_gradient) #subtracting because we want to move in the opposite direction of the gradient
    #this is because we want to minimize function
    beta1 = beta1 - (step_size * beta1_gradient)
    
    return [beta0, beta1]



def gradient_descent_iterator(y, x, beta0, beta1, step_size=.0001, iterations=500):
    
    mean_squared_errors = []
    mean_squared_errors.append(mean_squared_error(y, x, beta0, beta1))
    
    beta0s = [beta0]
    beta1s = [beta1]
    
    for i in range(iterations):
        [beta0, beta1] = gradient_update(y, x, beta0, beta1, step_size)
        mean_squared_errors.append(mean_squared_error(y, x, beta0, beta1))
        beta0s.append(beta0)
        beta1s.append(beta1)
        
    return [mean_squared_errors, beta0s, beta1s]



# format for runnning the above 3 functions:
    
x = np.random.random_sample(100)*100
y = x + np.random.normal(np.random.normal(0,15), 30, size=100) + 100

plt.figure(figsize=(10,8))

plt.scatter(x, y, s=70, c='steelblue')

plt.show()



### Plotting functions from earlier (nonessential)



def plot_regression(x, y, model):
    plt.figure(figsize=(10,8))
    axes = plt.gca()
    
    intercept = model.params[0]
    slope = model.params[1]

    for x_, y_ in zip(x, y):    
        plt.plot((x_, x_), (y_, x_*slope + intercept),
                 'k-', ls='dashed', lw=1)
        
    plt.scatter(x, y, s=70, c='steelblue')
    
    x_points = np.linspace(axes.get_xlim()[0], axes.get_xlim()[1], 100)
    
    regline_x = x_points
    regline_y = x_points*slope + intercept

    plt.plot(regline_x, regline_y, c='darkred', lw=3.5)

    plt.show()
    
    
def plot_leastsq_loss(model):
    plt.figure(figsize=(10,8))
    
    resids = model.resid
    
    resid_lim = np.max([abs(np.min(resids)), abs(np.max(resids))]) + 1
    
    resid_points = np.linspace(-1*resid_lim, resid_lim, 200)
    
    for r in resids:
        plt.plot((r, r), (0, r**2), 'k-', ls='dashed', lw=2)
        
    plt.plot(resid_points, resid_points**2, c='gold', alpha=0.7)
    

def plot_residuals_ladloss(model):
    
    resids = model.resid
    
    resid_lim = np.max([abs(np.min(resids)), abs(np.max(resids))]) + 1
    
    resid_points = np.linspace(-1*resid_lim, resid_lim, 200)
    
    plt.figure(figsize=(10,8))
    
    for r in resids:
        
        plt.plot((r, r), (0, abs(r)), 'k-', ls='dashed', lw=1)
        
    plt.plot(resid_points, np.abs(resid_points), c='gold', alpha=0.7)
    
    
    
    
    
##REMINDER OF NEXT STEPS:
#from http://localhost:8888/notebooks/week03/loss-functions-regression-metrics-practice.ipynb
# 4: Choose a continuous response variable and predictor variable from the dataset
# 5: Choose a small subset of the predictor and response variables you chose
# 6. Build a least squares regression model predicting your response from your predictors
plot_regression(x, y, model)
7. Plot the least squares regression
8. Build a least absolute deviation quantreg model on the same sample
Plot the LAD regression

10. Calculate the RMSE and the MAE between you response and predicted response



In [ ]:
# FUll run-through of gradient descent iteration (redundant with some code above):
    
def func(x):    #this needs to be modified to whatever the specific function is
    if x <= 1:
        return 2 * x * x
    return 2

def gradient(x): #this needs to be the derivative of the func
    if x <= 1:
        return 4 * x
    return 0

def gradient_descent(x, l=0.1):
    vector = np.array(x)
    return vector - l * np.array(gradient(x))


def iterate(x0, n=10):
    xs = [x0]
    ys = [func(x0)]
    for i in range(n):
        x = gradient_descent(xs[-1], l=0.1)
        xs.append(x)
        ys.append(func(x))
    return xs, ys    


xs = np.arange(-2, 3, 0.1)
ys = map(func, xs)

plt.figure(figsize=(10,8))
plt.plot(xs, ys, alpha=0.5, ls='dashed')

# Start gradient descent at x = -1.5
xs2, ys2 = iterate(-1.5, n=100)
plt.scatter(xs2, ys2, c='r', s=100)

# Start gradient descent at x = 2; where does it go?
xs2, ys2 = iterate(2, n=100)
plt.scatter(xs2, ys2, c='y', s=300)

## Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

In [ ]:
# Plot the data and the best fit line
## The data
plt.scatter(X, y)
## The line / model
plt.plot(X, predictions)

plt.show()
print "r^2:", model.score(X, y)
print "RMSE:", mean_squared_error(ys, predictions)
print "MAE:", mean_absolute_error(ys, predictions)
print "Coefficients:", model.coef_, model.intercept_

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.